In [8]:
import sys
import subprocess
import pkg_resources

required = {'scipy', 'numpy', 'pandas', 'openpyxl', 'scikit-learn', 'numba', 'matplotlib','jupyter', 'nbconvert', 'plotly'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
import numba as nb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from sys import argv
import os
from scipy import stats
import plotly.graph_objects as go
from plotly.graph_objs import Candlestick
from numba import jit, types
from numba.typed import List

Idée à faire : 

1.   Max(VT - Payoff) + Minimize V0
2.   V0/S0 & MAX(VT - Payoff)
3.   Max(VT - Payoff)/(V0/S0)



In [9]:
file = 'data.xlsx'
df = [pd.read_excel(file, sheet_name = sheet) for sheet in ['CAC', 'DAX', 'IBEX35TR', 'IBEX', 'SX5E', 'SX5T', 'SMI']]
for i in range(len(df)):
    df[i] = df[i].drop(0)

In [10]:
df[0]

,DATES,DELTA_DAYS,PX_LAST,PX_HIGH,PX_LOW,PX_VOLUME,S1/S,D_S
1,2000-11-22,1,5944.70,6049.60,5937.37,47470800,0.977583,-136.32
2,2000-11-23,1,6053.04,6055.21,5933.08,37599950,1.018225,108.34
3,2000-11-24,1,6145.65,6162.88,5989.13,47878450,1.015300,92.61
4,2000-11-27,3,6171.33,6254.17,6167.45,40542910,1.004179,25.68
5,2000-11-28,1,6069.22,6172.77,6055.04,68980960,0.983454,-102.11
...,...,...,...,...,...,...,...,...
4778,2019-03-15,1,5405.32,5421.51,5351.88,220247735,1.010382,55.54
4779,2019-03-18,3,5412.83,5416.49,5401.47,73648562,1.001389,7.51
4780,2019-03-19,1,5425.90,5443.10,5403.58,72894413,1.002415,13.07
4781,2019-03-20,1,5382.66,5429.44,5382.66,83309887,0.992031,-43.24


In [11]:
prices = df[0]["PX_LAST"].to_numpy()

In [12]:
@nb.njit(nb.types.float64(nb.types.int64, nb.types.int64, nb.types.float64, nb.types.float64[:], nb.types.float64[:], nb.types.float64),fastmath = True, cache = True)
def g(t, T, K, Ku, Kd, x):
  if t >= T:
      return max(x - K, 0)
  else:
      lamda = (1 - Kd[t]) / (Ku[t] - Kd[t])
      return g(t+1, T, K, Ku, Kd, Ku[t] * x) * lamda + g(t+1, T, K, Ku, Kd, Kd[t] * x) * (1 - lamda)
@nb.njit(nb.types.float64(nb.types.int64, nb.types.int64, nb.types.float64, nb.types.float64[:], nb.types.float64[:], nb.types.float64),fastmath = True, cache = True)
def Teta(t, T, K, Ku, Kd, x):
  if t < T :
      return (g(int(t), int(T), float(K), Ku, Kd, float(Ku[t]*x)) - g(int(t), int(T), float(K), Ku, Kd, float(Kd[t]*x))) / ((Ku[t] - Kd[t])*x)
  else:
      return 0.0
#@nb.njit(nb.types.float64[:,:](nb.types.float64[:],nb.types.int64),fastmath = True, cache = True)
def get_lists(prices, p):
  n = len(prices)
  result = []
  for i in range(n, n-p, -1):
      result.append(prices[:i])
  return result
def rolling_window2(prices, train_size, test_size):
  return np.array([np.array([np.array(prices[i:i+train_size]),np.array(prices[i+train_size:i+train_size+test_size])]) for i in range(len(prices) - train_size - test_size + 1)])
def rolling_window(prices, train_size, test_size):
  return np.array([np.array(prices[i:i+train_size+test_size]) for i in range(len(prices) - train_size - test_size + 1)])
def SuppFt(prices, ic):
  logged_prices = np.log(prices)
  m = np.mean(logged_prices)
  s = np.std(logged_prices)
  n = len(logged_prices)
  mu = m + (s**2)/2
  z = stats.t.ppf((1+ic)/2, n)
  err = (s**2/n) + (s**4/(2*(n-1)))
  interval = z * np.sqrt(err)
  SuppFt = np.array([np.exp(mu - interval), np.exp(mu + interval)]) / prices[-1]
  Ku = max(SuppFt[1],1)
  Kd = min(SuppFt[0],1)
  return Kd, Ku
#@nb.njit(nb.types.float64[:](nb.types.float64[:], nb.types.float64, nb.types.int64),fastmath = True)
#input [1,2,3,4,5,6,7,8,9,10] output => [[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8],...,[1,2,3,4,5]], au val max de  maturité <= p <=
#len(prices) >= T
def generate_suppFt(prices, ic, p):
  L = get_lists(prices, p)
  Supp_Ku = []
  Supp_Kd = []

  for l in L:
    t_ = SuppFt(l, ic)
    Supp_Kd.append(t_[0])
    Supp_Ku.append(t_[1])
  outp = np.array([Supp_Ku, Supp_Kd])
  return outp
#@nb.njit(nb.types.float64(nb.types.float64[:], nb.types.float64, nb.types.int64,nb.types.int64, nb.types.float64, nb.types.int64),fastmath = True)
def Qte(st, prices, ic, t, K, T):
  Supp = generate_suppFt(prices, ic, T)
  Supp_Ku = Supp[0]
  Supp_Kd = Supp[1]
  if t==0:
    V0 = g(int(t), int(T), float(K), np.array(Supp_Ku), np.array(Supp_Kd), float(st))
  else:
    V0 = 0
  qte = Teta(int(t), int(T), float(K), np.array(Supp_Ku), np.array(Supp_Kd), float(st))
  return qte, V0


In [24]:
L = [[1.5,5,3.8,4],[11,2,56.77445,45,777445.555,89895.332]]
typed_L = List()
for li in L:
  typed_L.append(List(np.array(li, dtype = np.dtype(float))))

#@nb.njit(cache = True)
def jitted(L):
  out = List()
  for i in range(len(L)):
    s=0
    for j in range(len(L[i])):
      s += L[i][j]
    out.append(float(s))
  return out

%timeit -n 10000 jitted(typed_L)

135 µs ± 8.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
#@jit(types.Tuple((types.List(types.float64), types.List(types.float64), types.List(types.List(types.float64)), types.List(types.List(types.float64)), types.List(types.float64)))(types.Array(types.float64, 1, 'C'),types.float64,types.int64,types.int64),nopython=True, fastmath=True, cache=True)
def simulation(data, ic, n_train, n_test):
  rollin = rolling_window(data, n_train,n_test)
  qte = []
  Vt = []
  V0 = []
  S0 = []
  Payoff = []
  VT = []
  for window in rollin:
    K = min(window)
    Qt_window = []
    Vt_window = []
    for i in range(n_train, len(window)):
      train = window[:i]
      qt, V0_window = Qte(window[i], train, ic, i - n_train, K, n_test)
      Qt_window.append(qt) #Avoir une trajectoire de quantités pour chaque window
      if i==n_train:
        V0.append(V0_window) #Avoir une liste des V0 pour chaque window
        Vt_ = V0_window
        S0.append(window[i])
      else:
        Vt_ = Vt_ + qt*(window[i] - window[i-1])
      Vt_window.append(Vt_) #Avoir une trajectoire de Vt pour chaque window
      if i==len(window) - 1 :
        payoff = max(window[i] - K, 0)
        Payoff.append(payoff) #créer une liste des Payoffs
    Vt.append(Vt_window) #Liste des trajectoires de Vt
    VT.append(Vt_window[-1]) #créer une liste des VT
    qte.append(Qt_window) #Liste des trajectoires des Quantités t
  return VT, Payoff, qte, Vt, V0, S0

In [26]:
n_train = 30 #durée du lookback
n_test = 10 #équivalent à la maturité de l'option
ic = 0.99 #confiance en %

In [27]:
VT, Payoff, qte, Vt, V0, S0 = simulation(prices, ic, n_train, n_test)

In [17]:
Valeur_portefeuille = VT
Payoff = Payoff

difference = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Valeur_portefeuille))]

candle_data = []
for i in range(len(Valeur_portefeuille)):
    if difference[i] > 0:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
    elif difference[i] < 0:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
    else:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille)), y=Valeur_portefeuille, name='Valeur portefeuille en T',
                                 line=dict(color='#339933'), marker=dict(color='#339933')),
                      go.Scatter(x=np.arange(0,len(Valeur_portefeuille)), y=Payoff, name='Payoff en T',
                                 line=dict(color='#9933CC'), marker=dict(color='#9933CC')),
                      go.Candlestick(x=np.arange(0,len(Valeur_portefeuille)), open=list(map(lambda x: x['open'], candle_data)), high=list(map(lambda x: x['high'], candle_data)), low=list(map(lambda x: x['low'], candle_data)), close=list(map(lambda x: x['close'], candle_data)), increasing_line_color= '#90EE90', decreasing_line_color= 'red', name = 'Différence VT - Payoff')])
fig.update_layout(title='Valeur portefeuille en T vs Payoff en T',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))


In [18]:
Valeur_portefeuille = VT
Payoff = Payoff
Diff = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Payoff))]


fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=Diff, name='VT - payoff en T',
                                 line=dict(color='#339933'), marker=dict(color='#339933'))])

fig.update_layout(title='VT - payoff en T',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()


In [19]:
Valeur_portefeuille = VT
Payoff =  Payoff
Diff = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Payoff))]
V0S0 = np.divide(V0, S0)

fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=Diff, name='VT - Payoff',
                                 line=dict(color='#339933'), marker=dict(color='#339933')),
                      go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=V0S0, name='V0/S0',
                                 line=dict(color='red'), marker=dict(color='red'))])

fig.update_layout(title='VT - Payoff % V0/S0',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()
print(sum(Diff))
print(sum(V0S0))

49411.629813627485
267.3409136032837


In [20]:

fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=V0, name='Prix V0',
                                 line=dict(color='#339933'), marker=dict(color='#339933'))])

fig.update_layout(title='Prix V0',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()


In [21]:

Valeur_portefeuille = VT
Payoff = Payoff
Diff = [100*(Valeur_portefeuille[i] - Payoff[i])/Payoff[i] if Payoff[i] != 0 else 0 for i in range(len(Payoff))]

fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=Diff, name='VT - payoff en % du Payoff en T',
                                 line=dict(color='#F08080'), marker=dict(color='#F08080'))])

fig.update_layout(title='VT - payoff en % du Payoff en T',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (%)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
# Set the font family and size
fig.update_layout(font=dict(family='Verdana', size=14))


# Set the color and width of the line
fig.update_traces(line=dict(color='#00BFFF', width=2))

# Set the color and font of the axis labels
fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
# Show the plot
fig.show()


$$c = \sqrt{a^2 + b^2}$$